In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x+'-0'
    elif len(splitted) == 2:
        return x
    else:
        raise ValueError('myerror')

In [9]:
def create_land_specs_df():
    basedir = './토지특성정보/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    df = pd.read_csv(basedir + filenames[0], encoding='euc-kr')#, usecols=[2,3,5,7,9,11,12,14,18,20,22])
    
    df['지번주소'] = df['법정동명'] + ' ' + df['지번'].apply(landnum_modifier)
    df.drop(columns=['법정동명', '지번'], inplace=True)
    
    return df.sort_values(['지번주소', '기준년도'])

In [10]:
land_specs_df = create_land_specs_df()
print(land_specs_df.shape)
land_specs_df.head()

(7468497, 25)


,고유번호,법정동코드,대장구분코드,대장구분명,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,지번주소
6755827,1168010300101000000,1168010300,1.0,일반,5961,2013,1,1,전,876.0,43,자연녹지지역,0,지정되지않음,510,전,3,완경사,7,부정형,12,맹지,325000,2017-10-17,서울특별시 강남구 개포동 100-0
6755828,1168010300101000000,1168010300,1.0,일반,5960,2014,1,1,전,876.0,43,자연녹지지역,0,지정되지않음,510,전,3,완경사,7,부정형,12,맹지,330000,2017-10-17,서울특별시 강남구 개포동 100-0
6755829,1168010300101000000,1168010300,1.0,일반,5954,2015,1,1,전,876.0,43,자연녹지지역,0,지정되지않음,510,전,3,완경사,7,부정형,12,맹지,335000,2017-10-17,서울특별시 강남구 개포동 100-0
6755830,1168010300101000000,1168010300,1.0,일반,5987,2016,1,1,전,876.0,43,자연녹지지역,0,지정되지않음,510,전,3,완경사,7,부정형,12,맹지,345000,2017-10-17,서울특별시 강남구 개포동 100-0
6755831,1168010300101000000,1168010300,1.0,일반,5964,2017,1,1,전,876.0,43,자연녹지지역,0,지정되지않음,510,전,3,완경사,7,부정형,12,맹지,355000,2018-05-23,서울특별시 강남구 개포동 100-0


In [11]:
land_specs_df['기준년도'].value_counts()

2013    954102
2014    946648
2015    940631
2016    939576
2017    930804
2018    926366
2019    920391
2020    909979
Name: 기준년도, dtype: int64

In [5]:
# 사이사이 비어있는 년도는 어떻게 할 것인지, 같은 주소지만 다른 대장구분코드를 갖고있는 주소는 어떻게 할 것인지.

In [13]:
main_data_df = pd.read_csv('main_data_ver_2.3.csv')

In [14]:
selected_df = land_specs_df[land_specs_df['지번주소'].isin(main_data_df['지번주소'].unique())].reset_index(drop=True)

In [15]:
selected_df.shape

(870231, 25)

In [16]:
selected_df['토지이동상황'].value_counts()

다세대     520680
단독       83533
연립       53356
아파트      49756
주상용      42349
주상기타     39618
주거기타     23444
상업기타     21855
상업용      11752
주거나지     10362
업무용       8261
자연림       1785
상업나지       837
공업기타       635
도로등        541
주상나지       533
공업용        382
토지임야       147
공원등         70
주차장등        58
임야기타        57
전           42
운동장등        40
전기타         38
공업나지        35
하천등         32
기타          24
위험시설         8
과수원          1
Name: 토지이동상황, dtype: int64

In [17]:
fruit_df = selected_df[selected_df['토지이동상황'] == '과수원']
fruit_df.head()

,고유번호,법정동코드,대장구분코드,대장구분명,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,지번주소
849883,1126010100200310012,1126010100,2.0,산,15184,2020,1,5,임야,1462.0,43,자연녹지지역,0,지정되지않음,520,과수원,3,완경사,4,사다리형,10,세로한면(불),136000,2020-08-12,서울특별시 중랑구 면목동 31-12


In [18]:
addr_df = selected_df[selected_df['지번주소'] == fruit_df['지번주소'].iloc[0]]
addr_df

,고유번호,법정동코드,대장구분코드,대장구분명,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,지번주소
849875,1126010100100310012,1126010100,1.0,일반,619,2013,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),1590000,2017-10-17,서울특별시 중랑구 면목동 31-12
849876,1126010100100310012,1126010100,1.0,일반,620,2014,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),1662000,2017-10-17,서울특별시 중랑구 면목동 31-12
849877,1126010100100310012,1126010100,1.0,일반,619,2015,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),1727000,2017-10-17,서울특별시 중랑구 면목동 31-12
849878,1126010100100310012,1126010100,1.0,일반,613,2016,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),1807000,2017-10-17,서울특별시 중랑구 면목동 31-12
849879,1126010100100310012,1126010100,1.0,일반,609,2017,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),1959000,2018-05-23,서울특별시 중랑구 면목동 31-12
849880,1126010100100310012,1126010100,1.0,일반,617,2018,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),2151000,2018-07-31,서울특별시 중랑구 면목동 31-12
849881,1126010100100310012,1126010100,1.0,일반,616,2019,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),2325000,2019-08-08,서울특별시 중랑구 면목동 31-12
849882,1126010100100310012,1126010100,1.0,일반,615,2020,1,8,대,245.0,14,제2종일반주거지역,0,지정되지않음,130,다세대,3,완경사,2,가로장방,8,세로한면(가),2488000,2020-08-12,서울특별시 중랑구 면목동 31-12
849883,1126010100200310012,1126010100,2.0,산,15184,2020,1,5,임야,1462.0,43,자연녹지지역,0,지정되지않음,520,과수원,3,완경사,4,사다리형,10,세로한면(불),136000,2020-08-12,서울특별시 중랑구 면목동 31-12


In [19]:
selected_df['대장구분명'].value_counts()

일반             866035
산                2716
가지번              1015
블럭지번(지구)          257
블럭지번              176
블럭지번(지구_롯트)        24
가지번(부번세분)           8
Name: 대장구분명, dtype: int64

In [23]:
# 일반, BL, 산, 가, 지구BL

def landtype_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if '가지' in x:
            return '가'
        elif ('블럭' in x) & ('지구' in x):
            return '지구블록'
        elif ('블럭' in x):
            return 'BL'
        else:
            return x
        

In [24]:
selected_df['대장구분명_test'] = selected_df['대장구분명'].apply(landtype_modifier)
selected_df['대장구분명_test'].value_counts()

일반      866035
산         2716
가         1023
지구블록       281
BL         176
Name: 대장구분명_test, dtype: int64

In [25]:
selected_df.head()

,고유번호,법정동코드,대장구분코드,대장구분명,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,지번주소,대장구분명_test
0,1168010300111630004,1168010300,1.0,일반,6817,2013,1,8,대,336.1,15,제3종일반주거지역,0,지정되지않음,230,상업나지,2,평지,3,세로장방,3,광대세각,6290000,2017-10-17,서울특별시 강남구 개포동 1163-4,일반
1,1168010300111630004,1168010300,1.0,일반,6814,2014,1,8,대,336.1,15,제3종일반주거지역,0,지정되지않음,240,상업기타,2,평지,3,세로장방,3,광대세각,6480000,2017-10-17,서울특별시 강남구 개포동 1163-4,일반
2,1168010300111630004,1168010300,1.0,일반,6808,2015,1,8,대,336.1,15,제3종일반주거지역,0,지정되지않음,240,상업기타,2,평지,3,세로장방,3,광대세각,6870000,2017-10-17,서울특별시 강남구 개포동 1163-4,일반
3,1168010300111630004,1168010300,1.0,일반,6860,2016,1,8,대,336.1,15,제3종일반주거지역,0,지정되지않음,240,상업기타,2,평지,3,세로장방,3,광대세각,7125000,2017-10-17,서울특별시 강남구 개포동 1163-4,일반
4,1168010300111630004,1168010300,1.0,일반,6835,2017,1,8,대,336.1,15,제3종일반주거지역,0,지정되지않음,240,상업기타,2,평지,3,세로장방,3,광대세각,7485000,2018-05-23,서울특별시 강남구 개포동 1163-4,일반


In [26]:
selected_df.rename(columns={'기준년도':'년'}, inplace=True)

In [27]:
selected_df['대장구분명'] = selected_df['대장구분명_test']
selected_df.drop(columns=['대장구분명_test'], inplace=True)

In [28]:
selected_df.drop_duplicates(subset=['지번주소', '대장구분명', '년'], keep='first', inplace=True)

In [29]:
selected_df = selected_df[['지번주소', '대장구분명', '년', '지목명', '토지면적', '용도지역명1', '용도지역명2', '토지이동상황',
                           '지형높이', '지형형상', '도로접면', '공시지가']]
print(selected_df.shape)
selected_df.head()

(868104, 12)


,지번주소,대장구분명,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 1163-4,일반,2013,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000
1,서울특별시 강남구 개포동 1163-4,일반,2014,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,6480000
2,서울특별시 강남구 개포동 1163-4,일반,2015,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,6870000
3,서울특별시 강남구 개포동 1163-4,일반,2016,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,7125000
4,서울특별시 강남구 개포동 1163-4,일반,2017,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,7485000


In [30]:
del land_specs_df, main_data_df

In [31]:
def create_complete_land_specs_df_iteratively(df):
    import numpy as np
    import pandas as pd
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        if (count%10000) == 0:
            gc.collect()
        
        addr_df = df[df['지번주소'] == addr]
        
        for code in addr_df['대장구분명'].unique():
            code_df = addr_df[addr_df['대장구분명'] == code]
            min_year = code_df['년'].min()
            
            years = [i for i in range(min_year, 2021)]
                        
            [years.remove(y) for y in code_df['년'].unique()]

            temp_dict = {}
            for col in df.columns:
                if col == '지번주소':
                    temp_dict[col] = addr
                elif col == '대장구분명':
                    temp_dict[col] = code
                elif col == '년':
                    temp_dict[col] = years
                else:
                    temp_dict[col] = np.nan

            add_df = pd.DataFrame(temp_dict)

            mini_concat_df = pd.concat([code_df, add_df]).sort_values(['년']).fillna(method='ffill').reset_index(drop=True)
        
            #min_year_row = mini_concat_df.loc[[mini_concat_df.index[min_year-2006]],:]
        
            #min_year_concat_df = pd.concat([min_year_row for i in range(min_year-2006)])
        
            #mini_concat_df.iloc[[i for i in range(min_year-2006)], 3:] = min_year_concat_df.iloc[:, 3:]
        
            dfs_list.append(mini_concat_df)
        
        count += 1

    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [32]:
def parallelize(df, func, num_processors=6):
    #data_split = np.array_split(data, num_processors)
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [33]:
%%time
completed_df = parallelize(selected_df, create_complete_land_specs_df_iteratively)
print(completed_df.shape)
completed_df.head()

(872183, 12)
Wall time: 3min 31s


,지번주소,대장구분명,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 1163-4,일반,2013.0,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0
1,서울특별시 강남구 개포동 1163-4,일반,2014.0,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,6480000.0
2,서울특별시 강남구 개포동 1163-4,일반,2015.0,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,6870000.0
3,서울특별시 강남구 개포동 1163-4,일반,2016.0,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,7125000.0
4,서울특별시 강남구 개포동 1163-4,일반,2017.0,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,7485000.0


In [35]:
completed_df['년'].value_counts()

2020.0    109457
2019.0    109411
2018.0    109292
2017.0    109121
2016.0    108990
2015.0    108842
2014.0    108617
2013.0    108453
Name: 년, dtype: int64

In [34]:
completed_df.to_csv('./prepped_data/land_specs_ver_2.csv', index=False)